In [22]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf
import PIL.ImageOps as ImageOps
import PIL.Image as Image
from tqdm import tqdm

In [23]:
class_names = ['Arive-Dantu',
                'Basale',
               'Betel',
               'Crape_Jasmine',
               'Curry', 'Drumstick',
               'Fenugreek',
               'Guava',
               'Hibiscus',
               'Indian_Beech',
               'Indian_Mustard',
               'Jackfruit',
               'Jamaica_Cherry-Gasagase',
               'Jamun',
               'Jasmine',
               'Karanda',
               'Lemon',
               'Mango',
               'Mexican_Mint',
               'Mint',
               'Neem',
               'Oleander',
               'Parijata',
               'Peepal',
               'Pomegranate',
               'Rasna',
               'Rose_apple',
               'Roxburgh_fig',
               'Sandalwood',
               'Tulsi']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (150,150)

In [24]:
def pre_process(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, IMAGE_SIZE) 
    return image

In [25]:
def load_data():
    
    datasets = ['C:/Users/sanju/OneDrive/Desktop/Nitheesh/Medicinal plant/Segmented','C:/Users/sanju/OneDrive/Desktop/Nitheesh/Test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = pre_process(img_path) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [26]:
(train_images, train_labels), (test_images, test_labels) = load_data()

Loading C:/Users/sanju/OneDrive/Desktop/Nitheesh/Medicinal plant/Segmented


100%|██████████| 500/500 [00:03<00:00, 140.06it/s]


Loading C:/Users/sanju/OneDrive/Desktop/Nitheesh/Test


100%|██████████| 99/99 [00:01<00:00, 67.55it/s]


In [29]:
n_train = train_labels.shape[0]
n_test = test_labels.shape[0]

print ("Number of training examples: {}".format(n_train))
print ("Number of testing examples: {}".format(n_test))
print ("Each image is of size: {}".format(IMAGE_SIZE))

Number of training examples: 14380
Number of testing examples: 2472
Each image is of size: (150, 150)


In [30]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

MemoryError: Unable to allocate 3.62 GiB for an array with shape (14380, 150, 150, 3) and data type float32

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(30, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)